In [1]:
import os
import tarfile
import urllib

In [2]:
DOWNLOAD_ROOT = "https://github.com/ageron/handson-ml2/tree/master"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

#### Extract CSV file from TGZ file